#### 카카오 API  #####

- 안내 페이지: http://bit.ly/pwd_kakaoAPI_guide
- 깃헙 주소: https://github.com/Play-with-data/datasalon

In [1]:
# folium 설치

#! pip install folium

# 지도시각화

###   데이터 준비

In [22]:
# 크롤링 데이터 불러오기
import pandas as pd
raw_total = pd.read_excel('./files/성수추천.xlsx')
raw_total.head()
raw_total=raw_total.dropna(axis=0)
raw_total.head(5)

,index,content,data,like,place,tags
1,1,.나가서 커피한잔 하면서멍때리면 딱 좋게...,2020-06-15,1525.0,"Seoul, Korea","['#로우커피스탠드', '#대한민국구석구석',..."
2,2,☁️성수동 골목에서 만난 외국감성 물씬 나ᄃ...,2020-07-02,3382.0,시몬스 하드웨어 스토어,"['#a7iii', '#sel2470gm']"
5,5,⠀보기만해도 시끌시끌한 유빈이네 주말도 끝 ...,2020-07-06,314.0,진해,[]
7,7,보고싶은 내동생 주찬이의 원픽💙🦋 ...이트...,2020-06-29,4215.0,연무장,[]
8,8,부지런히 살면 언젠가 보상을 받는다게으름을 피우면 당장 확실한 보상을 받는다,2020-07-06,254.0,대림 창고,[]


In [3]:
#위치정보 가져오기
location_counts = raw_total['place'].value_counts( )
location_counts

성수동                      54
Arco  아르코                38
할아버지공장                   34
Seoul, Korea             30
서울숲 Seoul Forest Park    26
                         ..
Dorrell 6z4               1
뚝섬역4번출구                   1
롯데백화점본점                   1
성수동 빌런《villain》           1
ARBY 알비                   1
Name: place, Length: 346, dtype: int64

In [4]:
# 등록된 위치정보별 빈도수 데이터
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.head()

,place
성수동,54
Arco 아르코,38
할아버지공장,34
"Seoul, Korea",30
서울숲 Seoul Forest Park,26


In [5]:
#위치정보 빈도수 데이터 저장하기
location_counts_df.to_excel('./files/성수추천count.xlsx')

In [24]:
#위치정보 종류 확인하기
locations = list( location_counts.index )
locations[:10]

['성수동',
 'Arco  아르코',
 '할아버지공장',
 'Seoul, Korea',
 '서울숲 Seoul Forest Park',
 '연무장',
 '성수동 성수2가',
 'ᴅʏᴜʟ ᴄᴏғғᴇᴇ 디열커피',
 '헤이보울 hey.bowl',
 '옹근달']

### 카카오 지도 API를 활용한 장소 검색

In [25]:
#카카오 검색 API
import requests
searching = '성수동' # 검색할 키워드
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)

headers = {
    #카카오API키입력
    "Authorization": "KakaoAK c365fabe892788f00e612734bd99c75f"
    # 입력시 반드시  KakaoAK 뒤에 한 칸 띄우고 API키 값을 적어야 합니다. 
}
places = requests.get(url, headers = headers).json()['documents']
places[:1]

[{'address_name': '서울 성동구 성수동1가 685-696',
  'category_group_code': 'AT4',
  'category_group_name': '관광명소',
  'category_name': '여행 > 관광,명소 > 테마파크',
  'distance': '',
  'id': '1683241129',
  'phone': '02-2261-1393',
  'place_name': '브릭캠퍼스 서울',
  'place_url': 'http://place.map.kakao.com/1683241129',
  'road_address_name': '서울 성동구 서울숲2길 32-14',
  'x': '127.04290967796628',
  'y': '37.545577319610814'}]

In [8]:
# 장소 검색 함수 만들기
def find_places(searching):
    # ① 접속URL 만들기
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    # ② headers 입력하기
    headers = {
    #카카오API키입력
    "Authorization": "KakaoAK c365fabe892788f00e612734bd99c75f" 
    # 입력시 반드시  KakaoAK 뒤에 한 칸 띄우고 API키 값을 적어야 합니다. 
    }
    # ③ API 요청&정보 받기
    places = requests.get(url, headers = headers).json()['documents']
    # ④ 필요한 정보 선택하기
    place = places[0] 
    name = place['place_name']
    x=place['x']
    y=place['y']
    data = [name, x, y, searching] 

    return data

In [9]:
from tqdm import tqdm_notebook

In [26]:
# 인스타그램 위치명 위치정보 검색하기
#### 검색시 시간이 많이 걸려, 현재 작업 진행현황을 볼 수 있도록,  tqdm_notebook 추가 했습니다.
import time
locations_inform = [ ]
for location in tqdm_notebook(locations):
    try:
        data = find_places(location)       
        locations_inform.append(data) 
        time.sleep(0.5) 
    except:
        pass
locations_inform[:4]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


[['브릭캠퍼스 서울', '127.04290967796628', '37.545577319610814', '성수동'],
 ['아르코커피', '127.053429525035', '37.5448882689356', 'Arco  아르코'],
 ['할아버지공장', '127.054835558322', '37.5410980207173', '할아버지공장'],
 ['페이스뮤직', '126.915848984951', '37.5512791744838', 'Seoul, Korea']]

In [28]:
#위치정보 저장하기
locations_inform_df = pd.DataFrame(locations_inform)
locations_inform_df.columns = ['name_official','경도','위도','인스타위치명']
locations_inform_df.to_excel('./files/위도경도.xlsx', index=False) # 위도경도 정보 액셀파일로 저장

In [29]:
#인스타 게시량 및 위치정보 데이터 불러오기
location_counts_df = pd.read_excel('./files/성수추천count.xlsx', index_col = 0)
locations_inform_df = pd.read_excel('./files/위도경도.xlsx')

In [30]:
#위치 데이터 병합하기(join)
location_data = pd.merge(locations_inform_df, location_counts_df, 
                         how = 'inner', left_on = 'name_official', right_index=True)

In [31]:
#동일한 name_official 가지는 데이터 --> 인스타 게시 횟수 합치기
location_data = location_data.pivot_table(index = ['name_official','경도','위도'], values = 'place', aggfunc='sum')
# place 로 정렬하기(내림차순)
location_data=location_data.sort_values(by='place' ,ascending=False )
location_data.head(5)

,,,place
name_official,경도,위도,
할아버지공장,127.054836,37.541098,34
옹근달,127.052990,37.542713,18
서울숲,127.037618,37.544322,16
성수연방,127.056959,37.541397,13
우란문화재단,127.054281,37.543764,10


In [32]:
#병합한 데이터 저장하기
location_data.to_excel('./files/성수result.xlsx')

### folium을 이용한 지도 시각화 ① - 개별 표시

In [33]:
#데이터 불러오기
location_data = pd.read_excel('./files/성수result.xlsx')
location_data.head()

,name_official,경도,위도,place
0,할아버지공장,127.054836,37.541098,34
1,옹근달,127.052990,37.542713,18
2,서울숲,127.037618,37.544322,16
3,성수연방,127.056959,37.541397,13
4,우란문화재단,127.054281,37.543764,10


In [34]:
#지도 표시하기
import folium

# 처음 나타나는 위도 경도 설정
sungsu =[37.541098, 127.054836]
map_sungsu = folium.Map(location = sungsu, zoom_start = 11)

for i in range(len(location_data)):
    name = location_data ['name_official'][i]    # 공식명칭
    count = location_data ['place'][i]           # 게시글 개수
    size = int(count)*2
    long = float(location_data['위도'][i])      
    lat = float(location_data['경도'][i])       
    folium.CircleMarker((long,lat), radius = size, color='red', popup=name).add_to(map_sungsu)
    
map_sungsu

In [18]:
#지도 저장하기
map_sungsu.save('./files/성수지도.html') 

### folium을 이용한 지도 시각화 ② - 그룹으로 표시

In [19]:
# 지도 표시하기(마커 집합)
from folium.plugins import MarkerCluster
from matplotlib import font_manager, rc
# 한글 폰트 설정
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

locations = []
names = []
for i in range(len(location_data)):
    data = location_data.iloc[i]  # 행 하나씩
    locations.append((float(data['위도']),float(data['경도'])))    # 위도 , 경도 순으로..
    names.append(data['name_official'])

icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(30, 30)
    });
}"""
# 처음 나타나는 위도 경도 설정
sungsu =[37.541098, 127.054836]
map_sungsu2 = folium.Map(location = sungsu, zoom_start = 11)
                       
marker_cluster = MarkerCluster(
    locations=locations, popups=names,
    name='성수',
    overlay=True,
    control=True,
    icon_create_function=icon_create_function
)

marker_cluster.add_to(map_sungsu2)
folium.LayerControl().add_to(map_sungsu2)

map_sungsu2


In [20]:
#지도 저장하기
map_sungsu2.save('./files/성수지도2.html') 